In [ ]:
from api_neurotask import *
from baselines.metrics import get_R2
from baselines.decoders import *
from baselines.preprocessing_funcs import *
from bayes_opt import BayesianOptimization, UtilityFunction

In [ ]:
parquet_file_path = 'NeuroTask/2_1_Chowdhury_CObump.parquet'
df = load_and_filter_parquet(parquet_file_path,['A', 'I','F'])

In [ ]:
df10 = rebin(df,bin_size=20,reset=True)

In [ ]:
neurons = [col for col in df10.columns if col.startswith('Neuron')]
X_train_list = []
X_test_list = []
X_val_list = []

y_train_list = []
y_test_list = []
y_val_list = []

bins_before=5 #How many bins of neural data prior to the output are used for decoding
bins_current=1 #Whether to use concurrent time bin of neural data
bins_after=0 #How many bins of neural data after the output are used for decoding

training_range=[0, 0.7]
testing_range=[0.8, 1]
valid_range=[0.7,0.8]

In [ ]:
# Iterate over each unique animal in the dataset
for a in df10['animal'].unique():
    # Select data for the current animal
    d = df10[(df10['animal'] == a)]

    # Calculate the total number of sessions for the current animal
    n_samples = d['session'].value_counts().max() 

    # Iterate over each session for the current animal
    for session in d['session'].unique():

        # Select data for the current session and filter out zero columns
        df_session = df10[(df10['animal'] == a) & (df10['session'] == session)][neurons].dropna(axis=1)
        df_session = df_session.loc[:, (df_session != 0).any(axis=0)]

        # Extract hand/cursor velocity data for the current session
        y = np.array(df10[ (df10['session'] == session) & (df10['animal'] == a)][['hand_vel_x', 'hand_vel_y']])
        
        # Convert DataFrame to NumPy array and transpose
        session_data = df_session.to_numpy()  # Transpose to have (n_features, n_samples)

        # Format for recurrent neural networks (SimpleRNN, GRU, LSTM)
        # Function to get the covariate matrix that includes spike history from previous bins
        X=get_spikes_with_history(session_data,bins_before,bins_after,bins_current)
        num_examples=X.shape[0]

        #Note that each range has a buffer of"bins_before" bins at the beginning, and "bins_after" bins at the end
        #This makes it so that the different sets don't include overlapping neural data
        training_set=np.arange(int(np.round(training_range[0]*num_examples))+bins_before,int(np.round(training_range[1]*num_examples))-bins_after)
        testing_set=np.arange(int(np.round(testing_range[0]*num_examples))+bins_before,int(np.round(testing_range[1]*num_examples))-bins_after)
        valid_set=np.arange(int(np.round(valid_range[0]*num_examples))+bins_before,int(np.round(valid_range[1]*num_examples))-bins_after)

        #Get training data
        X_train=X[training_set,:,:]
        y_train=y[training_set,:]

        #Get testing data
        X_test=X[testing_set,:,:]
        y_test=y[testing_set,:]

        #Get validation data
        X_valid=X[valid_set,:,:]
        y_valid=y[valid_set,:]

        #Z-score "X" inputs. 
        X_train_mean=np.nanmean(X_train,axis=0)
        X_train_std=np.nanstd(X_train,axis=0)
        X_train_std = np.where(X_train_std == 0, 1e-16, X_train_std)

        X_train=(X_train-X_train_mean)/X_train_std
        X_test=(X_test-X_train_mean)/X_train_std
        X_valid=(X_valid-X_train_mean)/X_train_std

        #Zero-center outputs
        y_train_mean=np.mean(y_train,axis=0)
        y_train=y_train-y_train_mean
        y_test=y_test-y_train_mean
        y_valid=y_valid-y_train_mean

        X_train_list.append(X_train)
        X_test_list.append(X_test)
        X_val_list.append(X_valid)
        

        y_train_list.append(y_train)
        y_test_list.append(y_test)
        y_val_list.append(y_valid)

In [ ]:
print("There are", sum(X.shape[2] for X in X_train_list), "unique neurons in the dataset in",len(X_train_list),"different sessions")

### Wiener Filter

In [ ]:
for i in range(len(X_train_list)):
    X_flat_train=X_train_list[i].reshape(X_train_list[i].shape[0],(X_train_list[i].shape[1]*X_train_list[i].shape[2]))

    X_flat_test = X_test_list[i].reshape(X_test_list[i].shape[0],(X_test_list[i].shape[1]*X_test_list[i].shape[2]))

    #Declare model
    model_wf=WienerFilterDecoder()

    #Fit model|
    model_wf.fit(X_flat_train,y_train_list[i])

    #Get predictions
    y_valid_predicted_wf=model_wf.predict(X_flat_test)

    #Get metric of fit
    R2s_wf=get_R2(y_test_list[i],y_valid_predicted_wf)
    print('session', i+1,'R2s:', R2s_wf)

### Wiener Cascade

In [ ]:
for i in range(len(X_train_list)):
    #for i in range(len(X_train_list)):
    X_flat_train=X_train_list[i].reshape(X_train_list[i].shape[0],(X_train_list[i].shape[1]*X_train_list[i].shape[2]))

    X_flat_test = X_test_list[i].reshape(X_test_list[i].shape[0],(X_test_list[i].shape[1]*X_test_list[i].shape[2]))

    #Declare model
    model_wf=WienerCascadeDecoder(degree=3)

    #Fit model
    model_wf.fit(X_flat_train,y_train_list[i])

    #Get predictions
    y_valid_predicted_wf=model_wf.predict(X_flat_test)

    #Get metric of fit
    R2s_wf=get_R2(y_test_list[i],y_valid_predicted_wf)
    print('session', i+1,'R2s:', R2s_wf)

### DNN

Hyperparameters optimization

In [ ]:
def dnn_evaluate(num_units,frac_dropout,n_epochs):
            num_units=int(num_units)
            frac_dropout=float(frac_dropout)
            n_epochs=int(n_epochs)
            model_dnn=DenseNNDecoder(units=num_units,dropout=frac_dropout,num_epochs=n_epochs)
            model_dnn.fit(X_train,y_train)
            y_valid_predicted_dnn=model_dnn.predict(X_valid)
            return np.mean(get_R2(y_valid,y_valid_predicted_dnn))

In [ ]:
for i in range(len(X_train_list)):   
        print('session ',i+1)   

        X_train = X_train_list[i].reshape(X_train_list[i].shape[0],(X_train_list[i].shape[1]*X_train_list[i].shape[2]))
        X_valid = X_val_list[i].reshape(X_val_list[i].shape[0],(X_val_list[i].shape[1]*X_val_list[i].shape[2]))
        y_train = y_train_list[i]
        y_valid = y_val_list[i] 
        
        #Do bayesian optimization
        dnnBO = BayesianOptimization(dnn_evaluate, {'num_units': (50, 500), 'frac_dropout': (0,.5), 'n_epochs': (2,200)})
        #lstmBO.maximize(init_points=20, n_iter=20, kappa=10)
        utility = UtilityFunction(kind="ucb", kappa=10, xi=0.0)
        dnnBO.maximize(init_points=10,n_iter=5)

        best_params=dnnBO.max['params']
        frac_dropout=float(best_params['frac_dropout'])
        n_epochs=int(best_params['n_epochs'])
        num_units=int(best_params['num_units'])

Test

In [ ]:
i = 0 #session

X_flat_train=X_train_list[i].reshape(X_train_list[i].shape[0],(X_train_list[i].shape[1]*X_train_list[i].shape[2]))

X_flat_test = X_test_list[i].reshape(X_test_list[i].shape[0],(X_test_list[i].shape[1]*X_test_list[i].shape[2]))

model_dnn=DenseNNDecoder(units=200,dropout=0.25,num_epochs=5)

#Fit model
model_dnn.fit(X_flat_train,y_train_list[i])

#Get predictions
y_test_predicted_dnn=model_dnn.predict(X_flat_test)

#Get metric of fit
R2s_dnn=get_R2(y_test_list[i],y_test_predicted_dnn)
print('session', i+1,'R2s:', R2s_dnn)

### LSTM

Hyperparameters optimization

In [ ]:
def lstm_evaluate(num_units,frac_dropout,n_epochs):
            num_units=int(num_units)
            frac_dropout=float(frac_dropout)
            n_epochs=int(n_epochs)
            model_lstm=LSTMDecoder(units=num_units,dropout=frac_dropout,num_epochs=n_epochs)
            model_lstm.fit(X_train,y_train)
            y_valid_predicted_lstm=model_lstm.predict(X_valid)
            return np.mean(get_R2(y_valid,y_valid_predicted_lstm))

In [ ]:
for i in range(len(X_train_list)):
        print('session ',i+1)
        X_train = X_train_list[i]
        X_valid = X_val_list[i]
        y_train = y_train_list[i]
        y_valid = y_val_list[i]
                
        #Do bayesian optimization
        lstmBO = BayesianOptimization(lstm_evaluate, {'num_units': (50, 500), 'frac_dropout': (0,.5), 'n_epochs': (2,25)})
        #lstmBO.maximize(init_points=20, n_iter=20, kappa=10)
        utility = UtilityFunction(kind="ucb", kappa=10, xi=0.0)
        lstmBO.maximize(init_points=10,n_iter=5)

        best_params=lstmBO.max['params']
        frac_dropout=float(best_params['frac_dropout'])
        n_epochs=int(best_params['n_epochs'])
        num_units=int(best_params['num_units'])

get the performance in test set

In [ ]:
#session id
i = 0

model_lstm=LSTMDecoder(units=210,dropout=0.37,num_epochs=23)

#Fit model
model_lstm.fit(X_train_list[i],y_train_list[i])

#Get predictions
y_test_predicted_lstm=model_lstm.predict(X_test_list[i])

#Get metric of fit
R2s_lstm=get_R2(y_test_list[i],y_test_predicted_lstm)
print('session', i+1,'R2s:', R2s_lstm)